# Thông Tin Sinh Viên
1. Họ tên: Võ Khoa Nam
2. Mssv: 19521877
3. Lớp: CS116.M12.KHCL

In [1]:
pip install geopandas

     |████████████████████████████████| 1.0 MB 4.3 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 6.3 MB 61.4 MB/s 


In [2]:
import geopandas
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


# BƯỚC 1

In [6]:
# Đọc file wine bằng geopandas
data = geopandas.read_file('/drive/MyDrive/Wine.csv')
data

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment,geometry
0,14.23,1.71,2.43,15.6,127,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065,1,None
1,13.2,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050,1,None
2,13.16,2.36,2.67,18.6,101,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185,1,None
3,14.37,1.95,2.5,16.8,113,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480,1,None
4,13.24,2.59,2.87,21,118,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.7,0.64,1.74,740,3,None
174,13.4,3.91,2.48,23,102,1.8,0.75,0.43,1.41,7.3,0.7,1.56,750,3,None
175,13.27,4.28,2.26,20,120,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835,3,None
176,13.17,2.59,2.37,20,120,1.65,0.68,0.53,1.46,9.3,0.6,1.62,840,3,None


# BƯỚC 2

In [29]:
# Chia data dữ liệu thành 13 trường dữ liệu X và 1 trường dữ liệu y
X = data.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12]]
y = data.iloc[:,[13]]

In [13]:
X.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline
0,14.23,1.71,2.43,15.6,127,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.2,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050
2,13.16,2.36,2.67,18.6,101,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.5,16.8,113,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480
4,13.24,2.59,2.87,21,118,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735


In [7]:
from sklearn.preprocessing import StandardScaler

In [15]:
# Chuẩn hoá đặc trưng đầu vào X bằng StandardScaler
scaler = StandardScaler().fit(X)

In [17]:
scaler_data = scaler.transform(X)

In [18]:
scaler_data 

array([[ 1.51861254, -0.5622498 ,  0.23205254, ...,  0.36217728,
         1.84791957,  1.01300893],
       [ 0.24628963, -0.49941338, -0.82799632, ...,  0.40605066,
         1.1134493 ,  0.96524152],
       [ 0.19687903,  0.02123125,  1.10933436, ...,  0.31830389,
         0.78858745,  1.39514818],
       ...,
       [ 0.33275817,  1.74474449, -0.38935541, ..., -1.61212515,
        -1.48544548,  0.28057537],
       [ 0.20923168,  0.22769377,  0.01273209, ..., -1.56825176,
        -1.40069891,  0.29649784],
       [ 1.39508604,  1.58316512,  1.36520822, ..., -1.52437837,
        -1.42894777, -0.59516041]])

# BƯỚC 3

In [19]:
from sklearn.decomposition import PCA

In [21]:
# Sử dụng PCA để giảm chiều dữ liệu từ 13 thành 2
pca = PCA(n_components=2)

In [26]:
pca.fit(scaler_data)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [23]:
x_pca = pca.transform(scaler_data)

In [27]:
# Kích thước data trước khi giảm 
scaler_data.shape

(178, 13)

In [28]:
# Kích thước data sao khi giảm 
x_pca.shape

(178, 2)

# BƯỚC 4

In [31]:
from sklearn.model_selection import train_test_split

In [35]:
# Chia tập dữ liệu ra thành 2 phần train và test 8:2
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size=0.2)

In [42]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(142, 2)
(36, 2)
(142, 1)
(36, 1)


# BƯỚC 5

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
# Dùng LogisticRegression phân lớp dữ liệu 
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
y_pred = log_reg.predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score
print('accuracy = ',accuracy_score(y_test, y_pred))

accuracy =  0.9722222222222222


# BƯỚC 6

In [53]:
from sklearn.model_selection import KFold

In [59]:
kf = KFold(n_splits=5, shuffle=True)

In [66]:
acc_l = []
for train_index, test_index in kf.split(X,y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  log_reg.fit(X_train,y_train)
  y_pred = log_reg.predict(X_test)
  acc_l.append(accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

In [73]:
sum = 0
for i in range(len(acc_l)):
  sum = sum + acc_l[i]

print("Accuracy trung bình là: ",sum/len(acc_l))

Accuracy trung bình là:  0.9496825396825397
